<a href="https://colab.research.google.com/github/DenyT17/Football_Object_Detection/blob/main/Football_Object_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip install ultralytics
import math
from typing import Generator
import matplotlib.pyplot as plt
import numpy as np
from ultralytics import YOLO
import cv2 as cv
from google.colab import drive
%matplotlib inline

drive.mount('/content/gdrive')

ROOT_DIR = '/content/gdrive/My Drive/Football_Detection'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [17]:
model = YOLO("/content/gdrive/My Drive/Football_Detection/runs/detect/train2/weights/best.pt")

In [16]:
video = "/content/gdrive/My Drive/Football_Detection/video/Why Teamwork is important in Football.mp4"


In [20]:
from __future__ import annotations
from dataclasses import dataclass, field
from typing import Tuple, Optional, List, Dict, Any
import os


def display_detections(image: np.ndarray, detections: List[Detection], thickness = 12) -> np.ndarray:
  new_img = image.copy()
  colors = [(255, 0, 0),(0, 255, 255),(255, 102, 0),(255, 0, 255)]
  for detection in detections:
    # Getting x and y coordination of objects
    x1, y1, x2, y2 = detection.rect
    # Getting confident and name of detected objects.
    conf = detection.confidence
    classnme = detection.class_name
    # Showing detected object and information about they
    cv.rectangle(image, (x1, y1), (x2, y2), colors[cls], thickness)
    cv.putText(img, f'{class_name}{conf}', (max(0, x1), max(40, y1 - 30)), 5, 1, colors[detection.class_id])
  return image
  

In [21]:

@dataclass
class Detection:
    rect: Rectangle
    class_id: int
    class_name: str
    confidence: float
    tracker_id: Optional[int] = None

    @classmethod
    def from_results(cls, pred: np.ndarray, names: Dict[int, str]) -> List[Detection]:
        result = []
        for x_min, y_min, x_max, y_max, confidence, class_id in pred:
            class_id=int(class_id)
            result.append(Detection(
                rect=Rectangle(
                    x=float(x_min),
                    y=float(y_min),
                    width=float(x_max - x_min),
                    height=float(y_max - y_min)
                ),
                class_id=class_id,
                class_name=names[class_id],
                confidence=float(confidence)
            ))
        return result

In [14]:
video_path = os.path.join(video)
video_path_out = '{}_out.mp4'.format(video_path)

cap = cv.VideoCapture(video_path)
ret, frame = cap.read()
H, W, _ = frame.shape
out = cv.VideoWriter(video_path_out, cv2.VideoWriter_fourcc(*'MP4V'), int(cap.get(cv2.CAP_PROP_FPS)), (W, H))
# Load a model
colors = [(255, 0, 0),(0, 255, 255),(255, 102, 0),(255, 0, 255)]
classNames = ['ball', 'goalkeeper', 'player', 'referee']
while ret:
    results = model(frame)[0]
    for r in results:
        boxes = r.boxes
        for box in boxes:
            # Getting x and y coordination of objects
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            # Getting confident and name of detected objects.
            conf = math.ceil((box.conf[0] * 100)) / 100
            cls = int(box.cls[0])
            # Showing detected object and information about they
            cv.rectangle(frame, (x1, y1), (x2, y2), colors[cls], 5)
            cv.putText(frame, f'{classNames[cls]}{conf}', (max(0, x1), max(40, y1 - 30)), 5, 1, colors[cls])
    out.write(frame)
    ret, frame = cap.read()
cap.release()
out.release()
cv.destroyAllWindows()


NameError: ignored

In [36]:
img = cv.imread("/content/gdrive/My Drive/Football_Detection/test/images/744b27_1_10_png.rf.322f9266079f0424f6aebb23c23cc6de.jpg")
prediction = model(img)
print(prediction.pred[0].cpu().numpy())
# detections = Detection.from_results(
#     pred=prediction[0], 
#     names=model.names)
# image = display_detections(img,detections,)
# plt.figure(figsize=(12, 12))
# plt.imshow(image[...,::-1])
# plt.show()


0: 640x640 1 goalkeeper, 19 players, 2 referees, 19.4ms
Speed: 0.8ms preprocess, 19.4ms inference, 2.2ms postprocess per image at shape (1, 3, 640, 640)


AttributeError: ignored